# Tabelas

> Functions to only request fipe auxiliary tables.

In [ ]:
#| default_exp tabelas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import json
import requests
import pandas as pd

base_url = "http://veiculos.fipe.org.br/api/veiculos/"

def consulta_tabela_referencia():
        url = f"{base_url}/ConsultarTabelaDeReferencia"
        req = requests.post(url)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json)
        df.columns = ['codigo_mes', 'mes']
        df['mes'] = df['mes'].str.strip()
        return df

def consulta_tabela_marcas(mes='agosto/2022', tipo_veiculo='carro'):
        url = f"{base_url}/ConsultarMarcas"

        tabela_referencia = consulta_tabela_referencia()
        codigo_tabela_referencia = int(tabela_referencia.query(f"mes == '{mes}'").loc[:,'codigo_mes'].values[0])

        tipos_veiculos = {'carro': 1, 'moto': 2, 'caminhao': 3}
        codigo_tipo_veiculo = tipos_veiculos[tipo_veiculo]

        body = {"codigoTabelaReferencia": codigo_tabela_referencia, "codigoTipoVeiculo": codigo_tipo_veiculo}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json).rename(columns={'Label': 'marca', 'Value': 'codigo_marca'})
        df['mes'] = mes

        return df

def consulta_tabela_modelos(mes='agosto/2022', tipo_veiculo='carro', codigo_marca=59):
        url = f"{base_url}/ConsultarModelos"
        tabela_referencia = consulta_tabela_referencia()
        codigo_tabela_referencia = int(tabela_referencia.query(f"mes == '{mes}'").loc[:,'codigo_mes'].values[0])

        tipos_veiculos = {'carro': 1, 'moto': 2, 'caminhao': 3}
        codigo_tipo_veiculo = tipos_veiculos[tipo_veiculo]

        body = {"codigoTabelaReferencia": codigo_tabela_referencia,
                "codigoTipoVeiculo": codigo_tipo_veiculo,
                "codigoMarca": codigo_marca}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json['Modelos']).rename(columns={'Label': 'modelo', 'Value': 'codigo_modelo'})
        return df

In [ ]:
consulta_tabela_referencia()

,codigo_mes,mes
0,288,agosto/2022
1,287,julho/2022
2,286,junho/2022
3,285,maio/2022
4,284,abril/2022
...,...,...
255,69,maio/2001
256,68,abril/2001
257,67,março/2001
258,63,fevereiro/2001


In [ ]:
consulta_tabela_marcas(mes='agosto/2022', tipo_veiculo='carro')

,marca,codigo_marca,mes
0,Acura,1,agosto/2022
1,Agrale,2,agosto/2022
2,Alfa Romeo,3,agosto/2022
3,AM Gen,4,agosto/2022
4,Asia Motors,5,agosto/2022
...,...,...,...
87,Troller,57,agosto/2022
88,Volvo,58,agosto/2022
89,VW - VolksWagen,59,agosto/2022
90,Wake,163,agosto/2022


In [ ]:
consulta_tabela_modelos(mes='agosto/2022', tipo_veiculo='carro', codigo_marca=59)

,modelo,codigo_modelo
0,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x2 Die,5585
1,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x4 Die,5586
2,AMAROK Comfor. 3.0 V6 TDI 4x4 Dies. Aut.,9895
3,AMAROK Comfor. CD 2.0 TDI 4x4 Dies. Aut.,8531
4,AMAROK CS2.0 16V/S2.0 16V TDI 4x2 Diesel,5748
...,...,...
516,VOYAGE SELEÇÃO 1.6 I MOTION T.Flex 8V 4p,6672
517,VOYAGE SELEÇÃO 1.6 Total Flex 8V 4p,6673
518,VOYAGE TREND 1.6 Mi Total Flex 8V 4p,4755
519,VOYAGE Trendline 1.0 T.Flex 12V 4p,7524
